In [1]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import pickle
import string

In [2]:
fund_name = 'seligman'

# CIK_LIST = ['100786', '881466', '316411', '817841', '750679', '726291', '750756', '1031594', '709146', '100802', '14358', '752372', '100852', '1238182', '813240', '793701', '857694', '1332239', '1109076', '1092633', '1142702', '862813', '1142702', '1086890', '1392577', '1238182', '1092633', '1086890', '1471420']

CIK_LIST = ['316411']



In [3]:
cur_wd = os.getcwd()
if 'trunk' in cur_wd:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output')
else:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output','working')
    
cached_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'cached_data')
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print('data directory')
cur_wd
print(cur_wd)
print('output directory')
print(output_directory)

print('cached data')
print(cached_wd)


data directory
data
output directory
output/working
cached data
cached_data


In [4]:
# print(os.path.dirname(cur_wd))

In [5]:
weblink = {}
txt = []

for CIK in CIK_LIST:
    weblink[CIK]  = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        txt.append(i)
        weblink[CIK].append(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK)+r"/"  + i)

In [6]:
with open(os.path.join(cached_wd, 'errors_dict.pickle'), 'rb') as handle:
    errors = pickle.load(handle)
    
print(errors)

{'867297': ['0001193125-04-202597.txt', '0001193125-05-115532.txt', '0001193125-05-232633.txt'], '1091439': ['0000950152-04-008622.txt', '0000950152-05-004867.txt', '0000950152-05-009599.txt'], '1301708': ['0000950152-05-009600.txt'], '1423227': [], '1098605': ['0000950136-05-006141.txt', '0000950136-04-003174.txt', '0000950136-05-001871.txt'], '1378240': ['0001104659-07-029038.txt', '0001104659-07-077264.txt', '0001104659-08-027201.txt', '0001104659-08-064591.txt'], '726735': ['0000950129-04-009378.txt', '0000950129-05-005810.txt', '0000950129-05-011455.txt'], '844779': ['0001193125-05-039836.txt', '0001193125-05-175146.txt', '0001193125-05-175147.txt', '0001193125-05-175620.txt'], '1398078': [], '774013': ['0001193805-05-000319.txt', '0001193805-05-001781.txt', '0001193805-05-002628.txt', '0001193805-05-002636.txt', '0001193805-05-002638.txt'], '1062806': ['0000900092-04-000310.txt', '0000900092-05-000103.txt', '0000900092-05-000135.txt', '0000900092-05-000453.txt', '0001193805-04-00

In [7]:
with open(os.path.join(cached_wd, 'panel2.pickle'), 'rb') as handle:
    panel2 = pickle.load(handle)
    

In [8]:
panel2 = panel2.loc[panel2['file_read'].isin(txt)]

In [9]:
print(panel2)
panel2.to_csv(os.path.join(output_directory, fund_name + '_panel2.csv'), sep = ',')

   CLASS-CONTRACT-ID                      CLASS-CONTRACT-NAME  \
0         C000026136      SELIGMAN GROWTH FUND, INC., CLASS A   
1         C000026137      SELIGMAN GROWTH FUND, INC., CLASS B   
2         C000026138      SELIGMAN GROWTH FUND, INC., CLASS C   
3         C000026139      SELIGMAN GROWTH FUND, INC., CLASS D   
4         C000026140      SELIGMAN GROWTH FUND, INC., CLASS I   
5         C000026141      SELIGMAN GROWTH FUND, INC., CLASS R   
0         C000026136      SELIGMAN GROWTH FUND, INC., CLASS A   
1         C000026137      SELIGMAN GROWTH FUND, INC., CLASS B   
2         C000026138      SELIGMAN GROWTH FUND, INC., CLASS C   
3         C000026139      SELIGMAN GROWTH FUND, INC., CLASS D   
4         C000026140      SELIGMAN GROWTH FUND, INC., CLASS I   
5         C000026141      SELIGMAN GROWTH FUND, INC., CLASS R   
0         C000026136      SELIGMAN GROWTH FUND, INC., CLASS A   
1         C000026137      SELIGMAN GROWTH FUND, INC., CLASS B   
2         C000026138     

## Test on One FILE

In [10]:

CIK  = '316411'
text =  '0000950123-10-053029.txt'


In [11]:
panel2.loc[panel2['file_read'] == text]

,CLASS-CONTRACT-ID,CLASS-CONTRACT-NAME,CLASS-CONTRACT-TICKER-SYMBOL,SERIES-ID,OWNER-CIK,SERIES-NAME,file_read,date_filed,company conformed name,reporting_date
0,C000026136,"SELIGMAN GROWTH FUND, INC., CLASS A",SGRFX,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
1,C000026137,"SELIGMAN GROWTH FUND, INC., CLASS B",SGBTX,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
2,C000026138,"SELIGMAN GROWTH FUND, INC., CLASS C",SGRCX,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
3,C000026140,"SELIGMAN GROWTH FUND, INC., CLASS R5",SGFIX,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
4,C000026141,"SELIGMAN GROWTH FUND, INC., CLASS R2",SGFRX,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
5,C000076414,"SELIGMAN GROWTH FUND, INC., NEW CLASS I",None,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
6,C000076415,"SELIGMAN GROWTH FUND, INC., CLASS R3",None,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331
7,C000076416,"SELIGMAN GROWTH FUND, INC., CLASS R4",None,S000009569,0000316411,"SELIGMAN GROWTH FUND, INC.",0000950123-10-053029.txt,20100526,SELIGMAN GROWTH FUND INC,20100331


In [12]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()
print(names)

['SELIGMAN GROWTH FUND, INC.']


In [13]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
html_table = urlopen(url).read()
soup = BeautifulSoup(html_table, "html.parser")
tags = soup.find_all(['tr','b'])

In [14]:
tables = []
iterate = 0

for h in tags:
    if 'seligman' in str(h).lower() and 'fund'  in str(h).lower() and '<b>' in str(h).lower():
        print(str(h).lower())
        
    try:
        df = pd.read_html(str(i), flavor="bs4")
        for j in df:
            if len(j.index) > 0 :
                tables.append(j)
                iterate = iterate + 1
    except:
        pass

<b>seligman growth fund, inc.</b>


In [15]:
def remove_junk(addline):
    addline = re.sub('<[^>]*>', ' ', addline)
    addline = re.sub('&nbsp;', ' ', addline)
    addline = re.sub('&#174;', ' ', addline)
    addline = re.sub('&#8480;', ' ', addline)
    addline = re.sub(r'&reg;',' ',addline)
    addline = re.sub(r'\n', ' ', addline)
    addline = re.sub(r'\t', ' ', addline)
    addline = re.sub('\xa0', ' ', addline)
    return addline

In [16]:
count = 0
for i in tables:
    s = 'table ' + str(count)
    print(s)
    print(i)
    count = count + 1

In [17]:
def get_holdings(df):
    filter_names = []
    cleaned_df = pd.DataFrame()
    add =0 
    if len(df.columns) >= 3:
        col_name = 0
        df = df.dropna(axis = 0, subset = [df.columns[col_name]])
        df = df.dropna(thresh = 3, axis = 0)
#         print(df)

        for count, row in df.iterrows():
            print(row[0])
#             print(row[0],row[1],row[2])
            name = str(row[0])
#             print(name)
            if name[-1] != r'%' and '(cost' not in name.lower() and 'issuer' not in name.lower() and 'security description' not in name.lower() \
            and 'due' not in name.lower() :
                keys = re.findall(r'\([^()]*\)',row[1])
                key = ''
                for k in keys:
                    if '-' not in k:
                        key = key + k
                        name = name.replace(k,'')
                        
#                         print(key)

                strip_punct = '!"#$%&\'()+,-./:;<=>?@[\\]^_`{|}'
                h = re.findall("[^A-Za-z0-9]", name.replace(' ', ''))
                key2 = ''.join(h)
                key2 = key2.strip(strip_punct)
                for c in key2:
                    name = name.replace(c,"")

                
                add = pd.DataFrame(df.loc[count]).dropna()
                add = add[~add.duplicated()]
                add = add.loc[1:]
#                 print(add)
#                 add = add.iloc[[0, -1]]
                for k in keys:
                    if '-' not in k:
                        add.iloc[1,0] = add.iloc[1,0].replace(k,'')
                    #remove commas from holdings, shares and holding value 
#                # all the length of add are same as 2:
                if len(add) == 2:
                    add = add.transpose().dropna()
                    add.columns = ['holdings shares', 'holdings value']
                    add['holdings name'] = name
                    add['key'] = key + key2
                    add.loc[0,'holdings shares'] = add.loc[0,'holdings shares'].replace('$','').replace(',','')
                    add.loc[0,'holdings value'] = add.loc[0,'holdings value'].replace('$','').replace(',','')
                    if add.loc[0,'holdings shares'].isdigit() and add.loc[0,'holdings value'].isdigit():
                        cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                        cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))

    return cleaned_df, add
    

In [18]:
def get_legend(df):
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    legend = pd.DataFrame(columns = ['code', 'symbol', 'restricted'])
    try:
        for count, row in df.iterrows():
            keys = str(row[0])
            code = str(row[1])
            if (keys.strip() != code.strip()) and (code.strip() != ''):
                if (len(keys) == 3 and '(' in keys and ')' in keys) or ((len(keys) == 1) and (keys.isdigit() is False) and (keys.isalpha() is False)):
                    if 'level' not in code.lower():
                        add = pd.DataFrame([code, keys, '']).T
                        add.columns = ['code', 'symbol', 'restricted']
                        if len(legend.index) == 0:
                            legend = add.copy()
                        else:
                            legend = pd.concat([legend, add], axis = 0,ignore_index = True, sort = False)
    except:
        pass

                    
    return legend

In [19]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()

holdings = pd.DataFrame()
legend = pd.DataFrame()
fundname = ''

for tr in tags:
    if 'seligman' in str(tr).lower() and 'fund'  in str(tr).lower() and '<b>' in str(tr).lower():
        s = str(tr).replace('&amp','and')
        s = s.replace('  ', '').replace(';','').replace(',','').replace('.','').strip()
        s = remove_junk(s)
        for n in names:
            if n.lower().replace(';','').replace(',','').replace('.','').strip() in str(s).lower().strip():
                fundname = n
#                 print(fundname)
                
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    row = list(filter(None, row))
    if len(row) >= 3:
        add = pd.DataFrame(row).T
        cleaned_df, add1 = get_holdings(add)
        if len(cleaned_df.index) > 0:
            cleaned_df.loc[:, 'fund name'] = fundname
            cleaned_df.loc[:, 'identifier'] = fundname
            if len(holdings.index) > 0:
                holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
            else:
                holdings = cleaned_df.copy()
#     print(cleaned_df)
                
    if len(row) == 2:
        add = pd.DataFrame(row).T
        add_legend = get_legend(add)
        if len(add_legend.index) > 0:
            add_legend.loc[:, 'identifier'] = fundname
            if len(legend.index) > 0:
                legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
            else:
                legend  = add_legend.copy()
        


Issuer
Goodrich Corp.
Honeywell International, Inc.
Precision Castparts Corp.
United Parcel Service, Inc., Class B
PepsiCo, Inc.
Amgen, Inc.
Dendreon Corp.
Genzyme Corp.
Gilead Sciences, Inc.
Vertex Pharmaceuticals, Inc.
Invesco Ltd.
The Goldman Sachs Group, Inc.
Celanese Corp., Series A
Monsanto Co.
Potash Corp of Saskatchewan, Inc.
Wells Fargo & Co.
Cisco Systems, Inc.
JDS Uniphase Corp.
QUALCOMM, Inc.
Apple, Inc.
EMC Corp.
Hewlett-Packard Co.
IBM Corp.
Issuer
Foster Wheeler AG
American Express Co.
Capital One Financial Corp.
Coinstar, Inc.
Qwest Communications International, Inc.
ABB Ltd., ADR
Halliburton Co.
Noble Corp.
Schlumberger Ltd.
Transocean Ltd.
Costco Wholesale Corp.
The Kroger Co.
Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Covidien PLC
Edwards Lifesciences Corp.
Medtronic, Inc.
Express Scripts, Inc.
WellPoint, Inc.
Cerner Corp.
NVR, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Amazon.com, Inc.
Baidu.com, Inc., ADR
Go

In [20]:
legend

,code,symbol,restricted,identifier
0,Securities are valued by using policies descri...,(a),,"SELIGMAN GROWTH FUND, INC."
1,Non-income producing.,(b),,"SELIGMAN GROWTH FUND, INC."
2,Foreign security values are stated in U.S. dol...,(c),,"SELIGMAN GROWTH FUND, INC."
3,"At March 31, 2010, security was partially or f...",(d),,"SELIGMAN GROWTH FUND, INC."
4,Affiliated Money Market Fund — The Fund may in...,(e),,"SELIGMAN GROWTH FUND, INC."
5,Represents fractional shares.,(f),,"SELIGMAN GROWTH FUND, INC."
6,"At March 31, 2010, the cost of securities for ...",(h),,"SELIGMAN GROWTH FUND, INC."
7,Industry classifications are identified in the...,(a),,"SELIGMAN GROWTH FUND, INC."
8,Money market fund that is a sweep investment f...,(b),,"SELIGMAN GROWTH FUND, INC."
9,Asset categories for Investments of Cash Colla...,(c),,"SELIGMAN GROWTH FUND, INC."


In [21]:
holdings.to_csv('check1.csv')

In [22]:
holdings['value multiplier'] = ''
holdings['textfile'] = text
holdings['CIK'] = CIK
holdings['date_filed'] = '' 
holdings['company conformed name'] = ''
holdings['reporting_date'] = ''

try:
    date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
    conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
    reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
except IndexError:
    date_filed = ''
    conformed_name = ''

holdings.loc[:, 'date_filed'] = date_filed
holdings.loc[:, 'company conformed name'] = conformed_name
holdings.loc[:, 'reporting_date'] = reporting_date

holdings.index = pd.RangeIndex(len(holdings.index))
print(holdings)

for z in weblink[CIK]:
    if text in z:
        matching_link = z

holdings['weblink'] = matching_link
legend['weblink'] = matching_link
legend['textfile'] = text

   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       10807998  United Parcel Service, Inc., ClassB       
2           548000       36255680                        PepsiCo, Inc.       
3            92400       15766212        The Goldman Sachs Group, Inc.       
4           206600        6580210              Celanese Corp., SeriesA       
5           245810       17555750                         Monsanto Co.       
6           484100       20327359                       QUALCOMM, Inc.       
7           858000       25851540                      Halliburton Co.       
8           441300        9558558                       The Kroger Co.       
9           341500       18987400                Wal-Mart Stores, Inc.       
10          130700        6983301                          Kellogg Co.       
11          464000       27004800           Baxter International

In [161]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") |legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'


Empty DataFrame
Columns: [code, symbol, restricted, identifier]
Index: []


In [162]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") | legend['code'].str.contains("private placement transaction"), 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    holdings.loc[(holdings['identifier'] == row['identifier']) & (holdings['textfile'] == row.textfile) \
                   & (holdings['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    
restricted_holdings = holdings.loc[(holdings['restricted'] == 'yes')]

Empty DataFrame
Columns: [code, symbol, restricted, identifier]
Index: []


In [172]:
def getpanel1(CIK, text, panel2):
    names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
    names = names.unique()

    holdings = pd.DataFrame()
    legend = pd.DataFrame()
    fundname = ''

    for tr in tags:
        if 'seligman' in str(tr).lower() and 'fund'  in str(tr).lower() and '<b>' in str(tr).lower():
            s = str(tr).replace('&amp','and')
            s = s.replace('  ', '').replace(';','').replace(',','').replace('.','').strip()
            s = remove_junk(s)
            for n in names:
                if n.lower().replace(';','').replace(',','').replace('.','').strip() in str(s).lower().strip():
                    fundname = n


        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td]
        row = list(filter(None, row))
        if len(row) == 3:
            add = pd.DataFrame(row).T
            cleaned_df, add1 = get_holdings(add)
            if len(cleaned_df.index) > 0:
                cleaned_df.loc[:, 'fund name'] = fundname
                cleaned_df.loc[:, 'identifier'] = fundname
                if len(holdings.index) > 0:
                    holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
                else:
                    holdings = cleaned_df.copy()

        if len(row) == 2:
            add = pd.DataFrame(row).T
            add_legend = get_legend(add)
            if len(add_legend.index) > 0:
                add_legend.loc[:, 'identifier'] = fundname
                if len(legend.index) > 0:
                    legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
                else:
                    legend  = add_legend.copy()
                    
                    
                    
    holdings['value multiplier'] = ''
    holdings['textfile'] = text
    holdings['CIK'] = CIK
    holdings['date_filed'] = '' 
    holdings['company conformed name'] = ''
    holdings['reporting_date'] = ''

    try:
        date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
        conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
        reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
    except IndexError:
        date_filed = ''
        conformed_name = ''

    holdings.loc[:, 'date_filed'] = date_filed
    holdings.loc[:, 'company conformed name'] = conformed_name
    holdings.loc[:, 'reporting_date'] = reporting_date

    holdings.index = pd.RangeIndex(len(holdings.index))
    print(holdings)

    for z in weblink[CIK]:
        if text in z:
            matching_link = z

    holdings['weblink'] = matching_link
    legend['weblink'] = matching_link
    legend['textfile'] = text
    
    return holdings, legend

    

In [164]:
holdings, legend = getpanel1('316411', '0000950123-10-053029.txt', panel2)

Issuer
Honeywell International, Inc.
United Parcel Service, Inc., Class B
PepsiCo, Inc.
The Goldman Sachs Group, Inc.
Celanese Corp., Series A
Monsanto Co.
QUALCOMM, Inc.
Issuer
Halliburton Co.
The Kroger Co.
Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted price

In [165]:
holdings.to_csv(os.path.join(output_directory, fund_name + '_test_holdings2.csv'), sep = ',')
legend.to_csv(os.path.join(output_directory, fund_name + '_test_legend2.csv'), sep = ',')
print(holdings)

   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       10807998  United Parcel Service, Inc., ClassB       
2           548000       36255680                        PepsiCo, Inc.       
3            92400       15766212        The Goldman Sachs Group, Inc.       
4           206600        6580210              Celanese Corp., SeriesA       
5           245810       17555750                         Monsanto Co.       
6           484100       20327359                       QUALCOMM, Inc.       
7           858000       25851540                      Halliburton Co.       
8           441300        9558558                       The Kroger Co.       
9           341500       18987400                Wal-Mart Stores, Inc.       
10          130700        6983301                          Kellogg Co.       
11          464000       27004800           Baxter International

# Run on all CIKs

In [173]:
panel1 = pd.DataFrame()


panel1 = pd.DataFrame()
panel1_legend = pd.DataFrame()
error_panel1 = []
no_holdings = {}

CIK_LIST = ['100786', '881466', '316411', '817841', '750679', '726291', '750756', '1031594', '709146', '100802', '14358', '752372', '100852', '1238182', '813240', '793701', '857694', '1332239', '1109076', '1092633', '1142702', '862813', '1142702', '1086890', '1392577', '1238182', '1092633', '1086890', '1471420']

for CIK in CIK_LIST:
    no_holdings[CIK] = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        if '.csv' not in i and i not in errors[CIK]:
            holdings, legend = getpanel1(str(CIK), str(i), panel2)
            print(holdings)
            if len(holdings) == 0:
                no_holdings[CIK].append(i)

            if panel1.empty:
                panel1 = holdings.copy()
                panel1_legend = legend.copy()
            else:
                panel1 = pd.concat([panel1, holdings], axis = 0, sort = False)
                panel1_legend = pd.concat([panel1_legend , legend], axis = 0, sort = False)

            panel1.index = pd.RangeIndex(len(panel1.index))


Issuer
Honeywell International, Inc.
United Parcel Service, Inc., Class B
PepsiCo, Inc.
The Goldman Sachs Group, Inc.
Celanese Corp., Series A
Monsanto Co.
QUALCOMM, Inc.
Issuer
Halliburton Co.
The Kroger Co.
Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted price

The Kroger Co.
Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           16

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

...

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

The Kroger Co.
Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           16

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

Wal-Mart Stores, Inc.
Kellogg Co.
Baxter International, Inc.
Medtronic, Inc.
The Procter & Gamble Co.
3M Co.
Issuer
AFLAC, Inc.
Prudential Financial, Inc.
Joy Global, Inc.
Target Corp.
Apache Corp.
Issuer
Pfizer, Inc.
CSX Corp.
Union Pacific Corp.
Intel Corp.
Oracle Corp.
Philip Morris International, Inc.
Total Common Stocks
(Cost: $1,392,069,365)
Total Money Market Fund
(Cost: $1,988,238)
(Cost: $462,242,519)
(Cost: $1,856,300,122)(h)
Fannie Mae REMICS
Total market value of collateral securities
Fannie Mae Grantor Trust
Total market value of collateral securities
Access Group Inc
Total market value of collateral securities
Fannie Mae Benchmark REMIC
Total market value of collateral securities
Unrealized appreciation
Unrealized depreciation
Net unrealized appreciation
quoted prices
in active
markets for
   holdings shares holdings value                        holdings name key  \
0           311757       14113239        Honeywell International, Inc.       
1           167800       1080

In [174]:
panel1.loc[:,'acq name'] = ''
panel1.loc[:,'acq date'] = ''
panel1.loc[:,'acq cost'] = ''
print(output_directory)
panel1.to_csv(os.path.join(output_directory, fund_name + '_panel1.csv'), sep = ',')
panel1_legend.to_csv(os.path.join(output_directory, fund_name + '_panel1_legend.csv'), sep = ',')

output/working


In [175]:

panel1_legend.loc[:,'restricted'] = 'no'

panel1_legend.loc[panel1_legend['code'].str.contains("restrict") | panel1_legend['code'].str.contains("level 3 security") \
              | panel1_legend['code'].str.contains("exempt from registration") |panel1_legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = panel1_legend.loc[panel1_legend['restricted'].str.contains("yes")]

restricted.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted_legend.csv'), sep = ',')
print(restricted)

panel1.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    try:
        panel1.loc[(panel1['identifier'] == row['identifier']) & (panel1['textfile'] == row.textfile) \
                       & (panel1['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    except:
        pass
    
restricted_holdings = panel1.loc[(panel1['restricted'] == 'yes')]
restricted_holdings.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted.csv'), sep = ',')

Empty DataFrame
Columns: [code, symbol, restricted, identifier, weblink, textfile]
Index: []
